# The Battle Of The Neighbourhood Project
## Using clustering and segmentation to determine which city is best for business

In [95]:
#Import all necessary libraries
import wget
import json 
import requests
import numpy as np 
import pandas as pd 
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

print('Libraries imported.')

Libraries imported.


In [2]:
# Web scrape neighbourhoods from wikipedia

table = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969', header = 0)

#Obtain the first table
df_toronto = table[0]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#df_toronto.rename(columns = {"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#Filter for Borough that do not have any assignment
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
#Merge the neighborhoods
df_toronto = df_toronto.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
#Change unassigned Neighborhood to a Borough's name
df_toronto.loc[85,'Neighbourhood'] = 'Queen\'s Park'

print (df_toronto.shape)

df_toronto.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
#Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlong = pd.read_csv("https://cocl.us/Geospatial_data")
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
latlong.rename(columns = {"PostalCode": "Postal Code"}, inplace = True)
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
latlong.shape

(103, 3)

In [25]:
latlong.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [26]:
#Merge the Two data frames i.e. the Latitude and Longitude dataframe & Neighborhoods dataframe
df_toronto.set_index("Postal Code")
latlong.set_index("Postal Code")
neighbor=pd.merge(df_toronto, latlong)
neighbor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [27]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
    )
)

Toronto has 11 boroughs and 103 neighborhoods.


In [48]:
#Use geopy library to get the latitude and longitude values of Toronto, Canada

address = 'Toronto,CA'

geolocator = Nominatim(user_agent="smy-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The Geograpical Co-ordinate of Seattle, Toronto are {}, {}.'.format(latitude, longitude))

The Geograpical Co-ordinate of Seattle, Toronto are 43.6534817, -79.3839347.


In [50]:
# create map of Toronto with latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# highlight markers on map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [51]:
scarborough_data = neighbor[neighbor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [52]:
scarborough_data.shape

(17, 5)

In [54]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="smy-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The Geograpical Co-ordinate of Scarborough, is {}, {}.'.format(latitude, longitude))

The Geograpical Co-ordinate of Scarborough, is 43.7729744, -79.2576479.


In [55]:
CLIENT_ID = 'TXOGBTASCURBWWUZMK5YFHEMWUAAKDS2XLI32AO2GS1YPRGK' # your Foursquare ID
CLIENT_SECRET = '1UNYSSH0DSP3H0WT2QOUFCRBRZ3HCLIPZR4REGC1HDTNHDQA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TXOGBTASCURBWWUZMK5YFHEMWUAAKDS2XLI32AO2GS1YPRGK
CLIENT_SECRET:1UNYSSH0DSP3H0WT2QOUFCRBRZ3HCLIPZR4REGC1HDTNHDQA


In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [58]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Found 89 venues in 17 neighborhoods.


In [59]:
print(scarborough_venues.shape)
scarborough_venues.head()

(89, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [60]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",9,9,9,9,9,9
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [62]:
print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

There are 80 distinct venues in 55 categories.


In [64]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [65]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.00,0.000,0.000000,0.0,0.250000,0.000000,0.0,0.0,0.00,0.000000,0.000,0.250000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.25,0.25,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000,0.00,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.25,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.00,0.125,0.125000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.125,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.125000,0.00,0.125,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.125000,0.000000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.00,0.000,0.071429,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000,0.071429,0.000000,0.00,0.071429,0.00,0.000000,0.000000,0.000000,0.142857,0.071429,0.000000,0.071429,0.00,0.000,0.00,0.0,0.000000,0.000000,0.071429,0.0,0.00,0.00,0.000000,0.0,0.000000,0.00,0.0,0.071429,0.000000,0.000000,0.071429,0.142857,0.000000,0.000000,0.000000,0.000000,0.071429,0.00,0.0,0.071429,0.000000,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.00,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.00,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.00,0.000,0.000000,0.0,0.000000,0.142857,0.0,0.0,0.00,0.000000,0.000,0.142857,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.000,0.00,0.0,0.285714,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.00,0.200,0.000000,0.0,0.000000,0.000000,0.2,0.1,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.1,0.000000,0.100000,0.000000,0.0,0.00,0.00,0.000000,0.1,0.000000,0.00,0.0,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000,0.00,0.000,0.111111,0.0,0.111111,0.000000,0.0,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.000000

In [66]:
num_top_venues = 10

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3         Chinese Restaurant  0.25
4        American Restaurant  0.00
5                  Pet Store  0.00
6         Italian Restaurant  0.00
7      Korean BBQ Restaurant  0.00
8             Medical Center  0.00
9              Metro Station  0.00


----Birch Cliff, Cliffside West----
                       venue  freq
0      General Entertainment  0.25
1               Skating Rink  0.25
2                       Café  0.25
3            College Stadium  0.25
4        American Restaurant  0.00
5                       Park  0.00
6         Italian Restaurant  0.00
7      Korean BBQ Restaurant  0.00
8  Latin American Restaurant  0.00
9                     Lounge  0.00


----Cedarbrae----
                       venue  freq
0           Hakka Restaurant  0.12
1                     Bakery  0.12
2                       Bank  0.12
3            T

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Breakfast Spot,Latin American Restaurant,Lounge,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Donut Shop
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Chinese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Gas Station,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,College Stadium,Furniture / Home Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Fast Food Restaurant,Pizza Place,Gas Station,Italian Restaurant,Thai Restaurant,Fried Chicken Joint,Shopping Mall,Bank,Convenience Store,Chinese Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Hakka Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Donut Shop,Discount Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Furniture / Home Store,Brewery,Pet Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Donut Shop
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Intersection,Bus Station,Soccer Field,Ice Cream Shop,Park,Metro Station,Convenience Store,Department Store
7,"Guildwood, Morningside, West Hill",Rental Car Location,Medical Center,Electronics Store,Bank,Donut Shop,Restaurant,Mexican Restaurant,Breakfast Spot,Intersection,Department Store
8,"Kennedy Park, Ionview, East Birchmount Park",Hobby Shop,Chinese Restaurant,Department Store,Coffee Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Donut Shop
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Donut Shop,Discount Store,Department Store


In [69]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                                        Bar
2nd Most Common Venue                      Vietnamese Restaurant
3rd Most Common Venue                         Chinese Restaurant
4th Most Common Venue                                Gas Station
5th Most Common Venue                     Furniture / Home Store
6th Most Common Venue                        Fried Chicken Joint
7th Most Common Venue                       Fast Food Restaurant
8th Most Common Venue                          Electronics Store
9th Most Common Venue                                 Donut Shop
10th Most Common Venue                            Discount Store
Name: 11, dtype: object

In [70]:
# set number of clusters
k = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# k-means clustering
kmeans = KMeans(n_clusters=k, random_state=2).fit(scarborough_grouped_clustering)

kmeans.labels_

array([4, 4, 4, 4, 0, 4, 4, 4, 4, 2, 1, 3, 1, 4, 4, 4], dtype=int32)

In [77]:
scarborough_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,4
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,4
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,4
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,2


In [78]:
#Note that the neighborhood Upper Rouge does not have any venues, so I will drop from dataset
scarborough_data.drop(scarborough_data[scarborough_data.Neighbourhood == 'Upper Rouge'].index, inplace = True)
#df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)

scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Donut Shop,Discount Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4,Bar,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Donut Shop,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Rental Car Location,Medical Center,Electronics Store,Bank,Donut Shop,Restaurant,Mexican Restaurant,Breakfast Spot,Intersection,Department Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Korean BBQ Restaurant,Pharmacy,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Gas Station,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,College Stadium,Furniture / Home Store


In [80]:
# create map
smap = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(smap)
       
smap

In [81]:
sb_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

sb_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

sb_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

In [82]:
sb_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,-79.239476,0,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Gas Station,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,College Stadium,Furniture / Home Store


In [83]:
sb_cluster_1

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Furniture / Home Store,Brewery,Pet Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Donut Shop
12,Scarborough,-79.262029,1,Chinese Restaurant,Breakfast Spot,Latin American Restaurant,Lounge,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Donut Shop


In [84]:
sb_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,-79.264848,2,General Entertainment,Skating Rink,Café,College Stadium,Chinese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [100]:
!conda install wget --y

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - wget


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1k             |       h9ed2024_0         2.2 MB
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  wget               pkgs/main/osx-64::wget-1.20.1-h051b688_0

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2021.5.3~ --> pkgs/main::ca-certificates-2021.7.5-hecd8cb5_1

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2021.5.30-py37hf~ --> pkgs/main::certifi-2021.5.30-py37hecd8cb5_0
  conda              conda-forge::conda-4.10.3-py37hf98548~ --> pkgs/main::conda-4.10.3-py3

In [101]:
!wget -q -O 'newyork_data.json' https://geo.nyu.edu/download/file/nyu-2451-34572-geojson.json

In [102]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [103]:
neighborhoods_data = newyork_data['features']

In [104]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [106]:
#Convert json to data frame
col_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=col_names)

In [107]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [108]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [110]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'smy-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [112]:
# create map of New York using latitude and longitude values
mapofnewyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapofnewyork)  
    
mapofnewyork

In [113]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [114]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent = 'smy-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [115]:
# create map of Manhattan using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

In [117]:
neighborhood_latitude = queens_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[10, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Long Island City are 40.75021734610528, -73.93920223915505.


In [124]:
LIMIT = 100
radius = 300
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=TXOGBTASCURBWWUZMK5YFHEMWUAAKDS2XLI32AO2GS1YPRGK&client_secret=1UNYSSH0DSP3H0WT2QOUFCRBRZ3HCLIPZR4REGC1HDTNHDQA&v=20180605&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=TXOGBTASCURBWWUZMK5YFHEMWUAAKDS2XLI32AO2GS1YPRGK&client_secret=1UNYSSH0DSP3H0WT2QOUFCRBRZ3HCLIPZR4REGC1HDTNHDQA&v=20180605&ll=TXOGBTASCURBWWUZMK5YFHEMWUAAKDS2XLI32AO2GS1YPRGK,1UNYSSH0DSP3H0WT2QOUFCRBRZ3HCLIPZR4REGC1HDTNHDQA&radius=20180605&limit=40.75021734610528'

In [119]:
results = requests.get(url).json()

In [125]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [137]:
dfnyc = pd.DataFrame(neighborhoods_data)
dfnyc.columns

Index(['type', 'id', 'geometry', 'geometry_name', 'properties'], dtype='object')